In [15]:
# Load libraries
library(tidyverse)
library(RPostgreSQL)

In [46]:
#Connect to database

host <- Sys.getenv("PG_HOST")
user <- Sys.getenv("PG_USER")
password <- Sys.getenv("PG_PASSWORD")

con <- dbConnect(RPostgres::Postgres(),
    dbname = "liberia_espen",
    host = host,
    port = 5432,
    user = user,
    password = password
)

# Read the views into data frames
child_view <- dbGetQuery(con, "SELECT * FROM v_espen_lr_sch_sth_impact_202401_2_child_v2_1")
kato_katz <- dbGetQuery(con, "SELECT * FROM v_espen_lr_sch_sth_impact_202401_3_kato_katz_v2_1")
urine_result_view <- dbGetQuery(con, "SELECT * FROM v_espen_lr_sch_sth_impact_202401_4_urine_v2_1")
metabase_dups <- dbGetQuery(con, "SELECT * FROM metabase_lr_sch_sth_202401_ia_duplicates")

# Close the database connection
dbDisconnect(con)

## Identify all duplicates

In [47]:
child_view_dups <- child_view %>%
    group_by(p_code_id) %>%
    filter(n() > 1) %>%
    select(id, p_code_id) %>%
    distinct()

kk_dups <- kato_katz %>%
  group_by(k_code_id) %>%
  filter(n() > 1) %>%
  select(id, k_code_id) %>%
  distinct()

urine_dups <- urine_result_view %>%
    group_by(u_code_id) %>%
    filter(n() > 1) %>%
    select(id, u_code_id) %>%
    distinct()

head(child_view_dups)
head(metabase_dups)


id,p_code_id
<int>,<chr>
1,168_9
100000000,168_9


id,id_participant,barcode_participant,id_results_kk,barcode_results_kk,id_results_urine,barcode_results_urine,form,status
<int>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>


## Load and update Metabase_dups

In [49]:
for (i in 1:nrow(child_view_dups)) {
  id <- child_view_dups$id[i]
  p_code_id <- child_view_dups$p_code_id[i]

  # Check if a matching record exists in Metabase_lr_sch_sth_202401_ia_duplicates
  matching_record <- metabase_dups %>%
    filter(id_participant == id, barcode_participant == p_code_id)

  if (nrow(matching_record) == 0) {
    # Insert a new record into Metabase_dups
    new_record <- tibble(
      # id = max(metabase_dups$id, na.rm = TRUE) + 1,
      id_participant = id,
      barcode_participant = p_code_id,
      status = 'Unsolved',
      # Other columns set to NULL
    )
    metabase_dups <- bind_rows(metabase_dups, new_record)
  }
}

# Update all remaining records in Metabase_lr_sch_sth_202401_ia_duplicates to 'Unsolved'
# Metabase_lr_sch_sth_202401_ia_duplicates %>%
#   left_join(Metabase_dups, by = c("id_participant" = "id", "barcode_participant" = "p_code_id")) %>%
#   mutate(status = ifelse(is.na(id_participant), 'Unsolved', status)) %>%
#   select(-id_participant, -barcode_participant) %>%
#   rename(id_participant = id, barcode_participant = p_code_id) -> metabase_dups

# Display the first few rows of Metabase_dups
head(metabase_dups)


,id,id_participant,barcode_participant,id_results_kk,barcode_results_kk,id_results_urine,barcode_results_urine,form,status
,<int>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>
1,NA,1,168_9,NA,NA,NA,NA,NA,Unsolved
2,NA,100000000,168_9,NA,NA,NA,NA,NA,Unsolved
3,NA,1,168_9,NA,NA,NA,NA,NA,Unsolved
4,NA,100000000,168_9,NA,NA,NA,NA,NA,Unsolved
